In [1]:
import os
from docx import Document

docs = "docs"

# Function to read .docx files in a directory

def read_docx(directory_path):
    # Initialize an empty list to store .docx files
    docx_files = []
    
    # Get all files in the directory
    all_files = os.listdir(directory_path)
    
    # Iterate through each file in the directory
    for file in all_files:
        # Check if the file ends with the .docx extension
        if file.endswith('.docx'):
            # If it does, add it to the list of .docx files
            docx_files.append(file)
    
    # Dictionary to store file names and their content
    docx_contents = []
    
    
    
    # Read each .docx file
    for file in docx_files:
        file_path = os.path.join(directory_path, file)
        try:
            # Open and read the document
            doc = Document(file_path)
            content = ' '.join([para.text for para in doc.paragraphs])
            
            docx_contents.append(content)
        except Exception as e:
            docx_contents[file] = f"Error reading file: {e}"
    
    return docx_contents

# Example usage (uncomment the line below to use with a specific directory)
read_docx(docs)




['Nuevo smartphone de alta gama promete revolucionar el mercado  Hoy se anunció el lanzamiento del último smartphone de la reconocida marca Tecnophone. Este nuevo dispositivo, denominado Tecnophone XZ, promete características sobresalientes que seguramente causarán revuelo en la industria móvil. Incorpora una pantalla OLED de 6.7 pulgadas, un procesador de última generación y una memoria RAM de 12 GB. Además, para los amantes de la fotografía, el Tecnophone XZ posee una cámara triple con una resolución de 108 megapíxeles. Los expertos ya lo han calificado como el "móvil del año" y esperan ver su impacto en el mercado en los próximos meses. ',
 'La duración de la batería en smartphones: un desafío constante  La vida útil de la batería se ha convertido en uno de los temas más discutidos en el mundo de los smartphones. Con la constante demanda de aplicaciones y servicios que consumen grandes cantidades de energía, los fabricantes están bajo presión para ofrecer soluciones innovadoras. Rec

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


def preprocess_text(text):
    
    puntucaction = '!"\#$%&\'\'()*+,-./:;<=>?@[\]^_``{|}~¿¡'
        
    text = text.lower()
    
    # Tokenize the text
    words = word_tokenize(text)
    
    words_clean = [word for word in words if word not in puntucaction]
   
    # Get the list of English stopwords
    stop_words = set(stopwords.words('spanish'))
    # Remove stopwords
    filtered_words = [word for word in words_clean if word not in stop_words]
    
    return filtered_words


tokenized_text = []
for text in read_docx(docs):
    tokenized_text.append(preprocess_text(text))
print(tokenized_text)

[['nuevo', 'smartphone', 'alta', 'gama', 'promete', 'revolucionar', 'mercado', 'hoy', 'anunció', 'lanzamiento', 'último', 'smartphone', 'reconocida', 'marca', 'tecnophone', 'nuevo', 'dispositivo', 'denominado', 'tecnophone', 'xz', 'promete', 'características', 'sobresalientes', 'seguramente', 'causarán', 'revuelo', 'industria', 'móvil', 'incorpora', 'pantalla', 'oled', '6.7', 'pulgadas', 'procesador', 'última', 'generación', 'memoria', 'ram', '12', 'gb', 'además', 'amantes', 'fotografía', 'tecnophone', 'xz', 'posee', 'cámara', 'triple', 'resolución', '108', 'megapíxeles', 'expertos', 'calificado', 'móvil', 'año', 'esperan', 'ver', 'impacto', 'mercado', 'próximos', 'meses'], ['duración', 'batería', 'smartphones', 'desafío', 'constante', 'vida', 'útil', 'batería', 'convertido', 'temas', 'discutidos', 'mundo', 'smartphones', 'constante', 'demanda', 'aplicaciones', 'servicios', 'consumen', 'grandes', 'cantidades', 'energía', 'fabricantes', 'bajo', 'presión', 'ofrecer', 'soluciones', 'innov

In [5]:
vocabulario_set = set()

for text in tokenized_text:
    for word in text:
        vocabulario_set.add(word)

vocabulario = list(vocabulario_set)
vectorized_text = []

for text in tokenized_text:
    doc_vector = []
    for word in vocabulario:
        count = text.count(word)
        doc_vector.append(count)
    vectorized_text.append(doc_vector)

print("Cantidad de textos vectorizados:", len(vectorized_text))


Cantidad de textos vectorizados: 12


In [6]:
# Imprimir el vocabulario
print("Vocabulario:", vocabulario)

# Imprimir los vectores generados junto con sus textos originales
for i, text in enumerate(tokenized_text):
    print(f"Texto original: {text}")
    print(f"Vectorizado: {vectorized_text[i]}")
    print("-" * 50)



Vocabulario: ['mobitech', 'precio', 'usuario', 'navegación', 'dispositivos', 'eléctricos', 'público', 'mercado', 'moderado', 'mejorar', 'disfrutar', 'ser', 'fitness', 'posibles', 'hoy', 'estratégicas', 'mejorando', 'ver', 'causarán', 'electrocar', 'mes', 'días', 'permitan', 'sistemas', 'contacto', 'mira', 'cómo', 'debido', 'amantes', 'haciendo', 'interés', 'disponibilidad', 'después', 'memoria', 'lanzada', 'permite', 'punto', 'gobierno', 'específico', 'expresarse', 'último', 'nuevo', 'aplicación', 'procesador', '108', 'fitjourney', 'financiera', 'lujo', 'capaces', 'realidad', 'cumple', 'dos', 'hito', 'medida', 'energía', 'constante', 'vehículo', 'última', 'adquisición', 'cero', 'ar', 'ofrecerá', 'diseño', 'panel', 'kilómetros', 'paradas', 'seguida', 'hacia', 'app', 'masiva', 'ofrece', 'tomar', 'calles', 'infinitas', 'servicio', 'lleva', 'tablet', 'automovilística', 'número', 'gran', 'innovación', '20', 'minutos', 'prometiendo', 'denominado', 'salto', 'compacto', 'podrán', 'tecnológicam